In [1]:
import pykep as pk
import heyoka as hy
import numpy as np
import pygmo as pg

The problem with the generalization of the Sims-Flanagan transcription to zero_hold is in the encoding of the throttles.
In essence the gradient w.r.t. the throttles becomes undefined when $u_x, u_y, u_z= 0$ 

In [2]:
# We build a ballistic leg
leg_sf = pk.leg.sims_flanagan_hf()
# We compute the graidents w.r.t. u
dmcdx0, dmcdxf, dmcdu = leg_sf.compute_mc_grad()
print(dmcdu)

[[        nan         nan         nan         nan         nan         nan
  -0.67822503]
 [        nan         nan         nan         nan         nan         nan
   0.67822503]
 [        nan         nan         nan         nan         nan         nan
   0.        ]
 [        nan         nan         nan         nan         nan         nan
  -0.67822503]
 [        nan         nan         nan         nan         nan         nan
  -0.67822503]
 [        nan         nan         nan         nan         nan         nan
   0.        ]
 [        nan         nan         nan         nan         nan         nan
   0.        ]]


This is bad as in low-thrust optimization, mass optimal problems have a bang-bang structure which will lead the optimizer to search in a neighborhood of the singular $u_x, u_y, u_z= 0$.

We avoid this singular behaviour regularizing our zero-hold system (non dimensional) as follows:

$$
   \left\{
   \begin{array}{l}
       \dot{\mathbf r} = \mathbf v \\
       \dot{\mathbf v} = -\frac{1}{r^3} \mathbf r + \frac{T}{m} \mathbf {\hat i} \\
       \dot m = - c T
   \end{array}\right.
$$

where $c = \frac 1 {I_{sp}g_0}$, $T$ is the thrust magnitude and $\mathbf {\hat i} = [i_x, i_y, i_z]$ the thrust direction. The right hand side of the EOMs now contains 4 control parameters $T$, $i_x, i_y, i_z$ with the added external constraint $|\mathbf {\hat i}| = 1$. 

In heyoka:

In [3]:
# The integrators (Keplerian)
tol=1e-14
tol_var = 1e-14

ta_global = pk.ta.get_zoh_kep(tol)
ta_var_global = pk.ta.get_zoh_kep_var(tol_var)


OK, now with the integrators built we can assemble a leg

In [4]:
class zoh_leg:
    def __init__(self, state0, controls, state1, tgrid, cut, tas = [ta_global, ta_var_global]):
        # We store the constructor args
        self.state0 = state0
        self.controls = controls
        self.state1 = state1
        self.tgrid = tgrid
        self.cut = cut
        # Store the tas
        self.ta = tas[0]
        self.ta_var = tas[1]
        # And save the non control parameter values for cfunc calls
        self.pars_no_control = self.ta.pars[4:].tolist()

        # We compute convenient quantities
        self.nseg = len(self.controls) // 4
        self.nseg_fwd = int(self.nseg * cut)
        self.nseg_bck = self.nseg - self.nseg_fwd

        # Sanity checks
        # On the tas
        if len(self.ta.state) != 7:
            raise ValueError(f"Attempting to construct a zoh_leg with a Taylor Adaptive integrator state dimension of {len(self.ta.state)}, while 7 is required")
        if len(self.ta.pars) <=4:
            raise ValueError(f"Attempting to construct a zoh_leg with a Taylor Adaptive integrator parameters dimension of {len(self.ta.pars)}, while >=4 is required")
        if len(self.ta_var.state) != 7 + 7*7 + 7*4:
            raise ValueError(f"Attempting to construct a zoh_leg with a variational Taylor Adaptive integrator state dimension of {len(self.ta_var.state)}, while 84 is required")
        if len(self.ta_var.pars) != len(self.ta.pars):
            raise ValueError(f"While constructing a zoh_leg, the number of parameters in the variational version of the Taylor integrator and the non variational version were detected as different, while its required they are equal")

        # On the rest        
        if len(self.controls) % 4 > 0:
            raise ValueError("In a zoh_leg controls must be multiple of 4: [T, ix, iy, iz] * nseg")
        if len(tgrid) != self.nseg + 1:
            raise ValueError("The t_grid and the controls have incompatible lenghts. It must be nseg*4 and nseg+1")

        # this assumes state of 7 and 4 pars
        self.ic_var = (np.hstack((np.eye(7,7), np.zeros((7,4))))).flatten()
        
        # We compile the function for the dynamics (this is used in the gradient computations)
        sys = self.ta.sys
        vars = [it[0] for it in sys]
        dyn = [it[1] for it in sys]
        self.dyn_cfunc = hy.cfunc_dbl(dyn, vars, compact_mode=True)

    def compute_mismatch_constraints(self):
        # Forward segments
        self.ta.time = self.tgrid[0]
        self.ta.state[:] = self.state0
        for i in range(self.nseg_fwd):
            # setting T, ix, iy, iz
            self.ta.pars[0:4] = self.controls[4*i:4*i+4]
            # propagating
            self.ta.propagate_until(self.tgrid[i+1])
        state_fwd = self.ta.state.copy()

        # Backward segments
        self.ta.time = self.tgrid[-1]
        self.ta.state[:] = self.state1
        for i in range(self.nseg_bck):
            # setting T, ix, iy, iz
            self.ta.pars[0] = self.controls[-4*i-4]
            self.ta.pars[1] = self.controls[-4*i-3]
            self.ta.pars[2] = self.controls[-4*i-2]
            self.ta.pars[3] = self.controls[-4*i-1]
            # propagating
            self.ta.propagate_until(self.tgrid[-2-i])
        state_bck = self.ta.state
        return (state_fwd - state_bck).tolist()
    
    def compute_throttle_constraints(self):
        retval = [0] * self.nseg
        for i in range(self.nseg):
            retval[i] = self.controls[1 + 4*i]**2 + self.controls[2 + 4*i]**2 + self.controls[3 + 4*i]**2 - 1
        return retval
            
    def compute_mc_grad(self):
        # We will return 4 matrices: dmc/dx0, dmc/dx1, dmc/dcontrol, dmc/dtgrid
        # STMs -> forward
        M_seg_fwd = [] # M10, M21, M32, ....
        M_fwd = [] # Mf0, Mf1, Mf2, ...
        C_seg_fwd = [] # dx_(i+1)/dcontrols_i
        C_fwd = np.zeros((7, 4*self.nseg_fwd)) # dx_f/dcontrols_i
        dyn_fwd = []
        self.ta_var.time = self.tgrid[0]
        self.ta_var.state[:7] = self.state0
        for i in range(self.nseg_fwd):
            self.ta_var.state[7:] = self.ic_var
            # setting T, ix, iy, iz
            self.ta_var.pars[0:4] = self.controls[4*i:4*i+4]
            # propagating
            self.ta_var.propagate_until(self.tgrid[i+1])
            # extracting the segment STMs
            M_seg_fwd.append(self.ta_var.state[7:].reshape(7,11)[:,:7].copy()) # 7x7
            # extracting the control sensitivities in across the single segment
            C_seg_fwd.append(self.ta_var.state[7:].reshape(7,11)[:,7:].copy()) # 7x4
            # computing the dynamics for future usage
            dyn_fwd.append(self.dyn_cfunc(self.ta_var.state[:7], pars = self.controls[4*i:4*i+4] + self.pars_no_control))
        # We compute the STMs - Mf0, Mf1, Mf2, ...
        cur = np.eye(7)  
        for M in reversed(M_seg_fwd):
            cur = cur @ M
            M_fwd.append(cur)
        M_fwd = list(reversed(M_fwd)) + [np.eye(7)]
        
        # 1 - dmc/dx0
        dmc_dx0 = M_fwd[0]
        
        # 2 - dmc/dcontrols
        i=0
        for M,C in zip(M_fwd[1:], C_seg_fwd):
            C_fwd[:,4*i:4*i+4] = (M@C)
            i+=1
        
        # 3 - dmc/dtgrid     
        dmcdtgrid = np.zeros((7,self.nseg+1))
        if self.nseg_fwd>0:
            dmcdtgrid[:,0] = -M_fwd[1]@dyn_fwd[0]
            dmcdtgrid[:,i] = M_fwd[-1]@dyn_fwd[-1]
            
            for i in range(1, self.nseg_fwd):
                dmcdtgrid[:,i]= M_fwd[i+1]@(M_seg_fwd[i]@dyn_fwd[i-1] - dyn_fwd[i])

        # STMs -> backward
        M_seg_bck = [] # M10, M21, M32, .... note: numbering reversed
        M_bck = [] # Mf0, Mf1, Mf2, .... note: numbering reversed
        C_seg_bck = [] # dxi/dcontrols
        C_bck = np.zeros((7, 4*self.nseg_bck))
        dyn_bck = []
        self.ta_var.time = self.tgrid[-1]
        self.ta_var.state[:7] = self.state1
        for i in range(self.nseg_bck):
            self.ta_var.state[7:] = self.ic_var
            # setting T, ix, iy, iz
            self.ta_var.pars[0] = self.controls[-4*i-4]
            self.ta_var.pars[1] = self.controls[-4*i-3]
            self.ta_var.pars[2] = self.controls[-4*i-2]
            self.ta_var.pars[3] = self.controls[-4*i-1]
            # propagating
            self.ta_var.propagate_until(self.tgrid[-2-i])
            # extracting the segment STMs
            M_seg_bck.append(self.ta_var.state[7:].reshape(7,11)[:,:7].copy())
            # extracting the control sensitivities in across the single segment
            C_seg_bck.append(self.ta_var.state[7:].reshape(7,11)[:,7:].copy()) # 7x4
            # computing the dynamics for future usage
            dyn_bck.append(self.dyn_cfunc(self.ta_var.state[:7], pars = self.controls[-4*i-4: -4*i-1] + [self.controls[-4*i-1]] + self.pars_no_control))
            
        # We compute the STMs - Mf0, Mf1, Mf2, ...
        cur = np.eye(7)  
        for M in reversed(M_seg_bck):
            cur = cur @ M
            M_bck.append(cur)
        M_bck = list(reversed(M_bck)) + [np.eye(7)]
        
        # 1 - dmc/dxf
        dmc_dx1 = -M_bck[0] # mc = xfwd-x_bck hence the minus
        
        # 2 - dmc/dcontrols
        # NOTE: the controls are reversed in order and we need to account for it
        i=0
        for M,C in zip(M_bck[1:], C_seg_bck):
            C_bck[:,4*self.nseg_bck-4-4*i:4*self.nseg_bck -4*i] = (M@C)
            i+=1
            
        # 3 - dmc/dtgrid 
        if self.nseg_bck>0: # we skip this in the corner case where no bck seg are there
            dmcdtgrid[:,-1] = M_bck[1]@dyn_bck[0]
            dmcdtgrid[:,self.nseg_fwd] -= M_bck[-1]@dyn_bck[-1] # This is for the mid time point shared fwd and bck: we need to subtract to the existing

            for i in range(1, self.nseg_bck):
                dmcdtgrid[:,-1-i]= -M_bck[i+1]@(M_seg_bck[i]@dyn_bck[i-1] - dyn_bck[i])     
        
        # dmc/dcontrols (7x4*nseg)
        dmc_dcontrols = np.hstack((C_fwd,-C_bck)) # mc = xfwd-x_bck hence the minus
        
        return dmc_dx0, dmc_dx1, dmc_dcontrols, dmcdtgrid
    
    # To compute numerical gradients its convenientto have this API
    def compute_mismatch_constraints_n(self, state0, controls, state1, tgrid):
        # Forward segments
        self.ta.time = tgrid[0]
        self.ta.state[:] = state0
        for i in range(self.nseg_fwd):
            # setting T, ix, iy, iz
            self.ta.pars[0:4] = controls[4*i:4*i+4]
            # propagating
            self.ta.propagate_until(tgrid[i+1])
        state_fwd = self.ta.state.copy()

        # Backward segments
        self.ta.time = tgrid[-1]
        self.ta.state[:] = state1
        for i in range(self.nseg_bck):
            # setting T, ix, iy, iz
            self.ta.pars[0] = controls[-4*i-4]
            self.ta.pars[1] = controls[-4*i-3]
            self.ta.pars[2] = controls[-4*i-2]
            self.ta.pars[3] = controls[-4*i-1]
            # propagating
            self.ta.propagate_until(tgrid[-2-i])
        state_bck = self.ta.state
        return (state_fwd - state_bck).tolist()

# We test the zero hold leg

In [5]:
# We create a ballistic leg 
t0 = 10000
t1 = 10400
pl0 = pk.planet(pk.udpla.jpl_lp("Earth"))
pl1 = pk.planet(pk.udpla.jpl_lp("Mars"))
r0, v0 = pl0.eph(t0)
r1, v1 = pl1.eph(t1)
l = pk.lambert_problem(r0=r0, r1=r1, tof = (t1-t0) * pk.DAY2SEC, mu = pk.MU_SUN)
m0 = 1000
m1 = 1000


# Instantiate the leg (ballistic so all T are 0)
# nd units
L = pk.AU
MU = pk.MU_SUN
TIME = np.sqrt(L**3/MU)
V =  L/TIME
ACC = V/TIME
MASS = 1000
F = MASS*ACC

# leg data
nseg = int(np.random.uniform(4, 20))
veff = np.random.uniform(2000, 7000) * pk.G0

# nd data
state0 = [it/L for it in r0] + [it/V for it in l.v0[0]] + [m0/MASS]
state1 = [it/L for it in r1] + [it/V for it in l.v1[0]] + [m1/MASS]
veff_nd = veff / V
tgrid = np.linspace(t0*pk.DAY2SEC/TIME, t1*pk.DAY2SEC/TIME, nseg+1)

uixiyiz = np.random.uniform(-1,1, (4*nseg,)) * 1e-3

cut = np.random.uniform(0,1)

ta_global.pars[4] = 1. / veff_nd
ta_var_global.pars[4] = 1. / veff_nd

leg = zoh_leg(state0, uixiyiz.tolist(), state1, tgrid, cut = cut, tas = [ta_global, ta_var_global])


In [6]:
# Chack on dmc/dx0
grad_an = leg.compute_mc_grad()[0]
grad_num = pg.estimate_gradient(lambda x: leg.compute_mismatch_constraints_n(x, leg.controls, leg.state1, leg.tgrid), leg.state0).reshape(7,-1)
np.linalg.norm(grad_num-grad_an)

1.0077049373830314e-08

In [7]:
# Chack on dmc/dxf
grad_an = leg.compute_mc_grad()[1]
grad_num = pg.estimate_gradient(lambda x: leg.compute_mismatch_constraints_n(leg.state0, leg.controls, x, leg.tgrid), leg.state1).reshape(7,-1)
np.linalg.norm(grad_num-grad_an)

6.0937194148521648e-07

In [8]:
# Chack on dmc/dcontrols
grad_an = leg.compute_mc_grad()[2]
grad_num = pg.estimate_gradient(lambda x: leg.compute_mismatch_constraints_n(leg.state0, x, leg.state1, leg.tgrid), leg.controls, dx=1E-7).reshape(7,-1)
np.linalg.norm(grad_num-grad_an)

8.9696023622118689e-08

In [9]:
grad_an = leg.compute_mc_grad()[3]
grad_num = pg.estimate_gradient(lambda x: leg.compute_mismatch_constraints_n(leg.state0, leg.controls, leg.state1, x), leg.tgrid).reshape(7,-1)
np.linalg.norm(grad_num-grad_an)

5.9342766855410302e-09

Now the gradient is always well defined, non nans:

In [10]:
leg = zoh_leg(state0, [0, 0.,0,0]*6, state1, np.linspace(0,400, 7)*pk.DAY2SEC/TIME, cut = cut, tas = [ta_global, ta_var_global])
dmcdx0, dmcdu, dmcdxf, dmdtgrid = leg.compute_mc_grad()
dmcdu

array([[ -7.20370308e+00,  -6.56137992e+00,  -3.14734807e-01,
          1.80455828e+01,   8.62151562e-01,   1.48460014e+00,
         -0.00000000e+00],
       [  8.41678545e+00,   9.67097898e+00,   2.27654828e-01,
         -2.46865848e+01,   2.74034306e+00,  -2.24706403e+00,
         -0.00000000e+00],
       [ -1.12942438e+00,  -1.04682601e+00,   6.50259827e-01,
          2.87424224e+00,  -7.31412566e-02,   2.95448472e-01,
         -0.00000000e+00],
       [ -4.62228783e+00,  -4.39142039e+00,  -1.90925857e-01,
          1.12575342e+01,   7.82915762e-01,   7.89408386e-01,
         -0.00000000e+00],
       [ -7.88545790e+00,  -9.19542044e+00,  -1.47495336e-01,
          2.29159305e+01,  -2.33396703e+00,   2.15732899e+00,
         -0.00000000e+00],
       [ -8.78820485e-04,   1.49809690e-01,   4.20458195e-01,
         -4.14446418e-01,   2.74047404e-01,   1.42948179e+00,
         -0.00000000e+00],
       [ -0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00,
         -0.00000000e+00,  -0.00